```markdown
![Sample Image](https://github.com/kuljotSB/EssencifAIconsultancy/blob/main/doc_intelli_flow/assets/prompt_flow_image.png?raw=true)
```

### Extract_Info component

In [ ]:
from promptflow import tool
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest


endpoint = "https://kuljotdocintelli.cognitiveservices.azure.com/"
key = "3XNgoPaZpLAVKobCXlAjOb6mUL66lAVIBSFKK3O7Cn85NcS1ei3EJQQJ99BCACYeBjFXJ3w3AAALACOGooKD"

# The inputs section will change based on the arguments of the tool function, after you save the code
# Adding type to arguments and return value will help the system show the types properly
# Please update the function name/signature per need
@tool
def document_intelligence(url: str)->dict:
  finalResponse=""

  document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    
  poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-invoice", AnalyzeDocumentRequest(url_source=url)
    )
  invoices = poller.result()

  invoice_data = []

  for idx, invoice in enumerate(invoices.documents):
    invoice_dict = {}
    
    def add_field(field_name, invoice_field, value_attr):
        if invoice_field:
            invoice_dict[field_name] = getattr(invoice_field, value_attr, None)
    
    add_field("VendorName", invoice.fields.get("VendorName"), "value_string")
    add_field("VendorAddress", invoice.fields.get("VendorAddress"), "value_address")
    add_field("VendorAddressRecipient", invoice.fields.get("VendorAddressRecipient"), "value_string")
    add_field("CustomerName", invoice.fields.get("CustomerName"), "value_string")
    add_field("CustomerId", invoice.fields.get("CustomerId"), "value_string")
    add_field("CustomerAddress", invoice.fields.get("CustomerAddress"), "value_address")
    add_field("CustomerAddressRecipient", invoice.fields.get("CustomerAddressRecipient"), "value_string")
    add_field("InvoiceId", invoice.fields.get("InvoiceId"), "value_string")
    add_field("InvoiceDate", invoice.fields.get("InvoiceDate"), "value_date")
    add_field("InvoiceTotal", invoice.fields.get("InvoiceTotal"), "value_currency.amount")
    add_field("DueDate", invoice.fields.get("DueDate"), "value_date")
    add_field("PurchaseOrder", invoice.fields.get("PurchaseOrder"), "value_string")
    add_field("BillingAddress", invoice.fields.get("BillingAddress"), "value_address")
    add_field("BillingAddressRecipient", invoice.fields.get("BillingAddressRecipient"), "value_string")
    add_field("ShippingAddress", invoice.fields.get("ShippingAddress"), "value_address")
    add_field("ShippingAddressRecipient", invoice.fields.get("ShippingAddressRecipient"), "value_string")
    
    invoice_dict["Items"] = []
    
    for item in invoice.fields.get("Items", {}).value_array:
        item_dict = {}
        add_field("Description", item.value_object.get("Description"), "value_string")
        add_field("Quantity", item.value_object.get("Quantity"), "value_number")
        add_field("Unit", item.value_object.get("Unit"), "value_number")
        add_field("UnitPrice", item.value_object.get("UnitPrice"), "value_currency.amount")
        add_field("ProductCode", item.value_object.get("ProductCode"), "value_string")
        add_field("Date", item.value_object.get("Date"), "value_date")
        add_field("Tax", item.value_object.get("Tax"), "value_string")
        add_field("Amount", item.value_object.get("Amount"), "value_currency.amount")
        invoice_dict["Items"].append(item_dict)
    
    add_field("SubTotal", invoice.fields.get("SubTotal"), "value_currency.amount")
    add_field("TotalTax", invoice.fields.get("TotalTax"), "value_currency.amount")
    add_field("PreviousUnpaidBalance", invoice.fields.get("PreviousUnpaidBalance"), "value_currency.amount")
    add_field("AmountDue", invoice.fields.get("AmountDue"), "value_currency.amount")
    add_field("ServiceStartDate", invoice.fields.get("ServiceStartDate"), "value_date")
    add_field("ServiceEndDate", invoice.fields.get("ServiceEndDate"), "value_date")
    add_field("ServiceAddress", invoice.fields.get("ServiceAddress"), "value_address")
    add_field("ServiceAddressRecipient", invoice.fields.get("ServiceAddressRecipient"), "value_string")
    add_field("RemittanceAddress", invoice.fields.get("RemittanceAddress"), "value_address")
    add_field("RemittanceAddressRecipient", invoice.fields.get("RemittanceAddressRecipient"), "value_string")
    
    invoice_data.append(invoice_dict)
  return invoice_data

  
 

### Summarisation Component

In [ ]:
#system:
You are a helpful AI assistant made to behave as a document extraction bot. 
Prior to calling you, the analyse API of azure document intelligence was called
to extract information from a document and you will be provided with the JSON schema
of the document information containing key-value pairs like merchant address, items,
subtotal value etc of a single invoice.

Your work is to generate a markdown of the information contained in the invoice 
passed down to you!

#user:
document extracted information: {{doc__information}}

### Output_Parser component

In [ ]:
from promptflow import tool
import json
from datetime import date

def serialize(obj):
    """Custom serialization function for unsupported types."""
    if isinstance(obj, date):  # Convert datetime.date to string
        return obj.isoformat()
    if hasattr(obj, "__dict__"):  # Convert objects with `__dict__` attribute to a dict
        return obj.__dict__
    return str(obj)  # Fallback to string conversion

@tool
def my_python_tool(markdown_text: str, json_struct: dict):
    final_response_dict = {
        "markdown_text": markdown_text,
        "json_struct": json_struct
    }

    return json.loads(json.dumps(final_response_dict, default=serialize))  # Ensure JSON serializable
